In [41]:
# imports
import numpy as np
import pandas as pd
import json
import gmaps
import requests
from pandas.io.json import json_normalize
import sys

from matplotlib import pyplot as plt
%matplotlib inline

from leuvenair.myutils.utility import readJson

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# read all the sensor latitudes and longitudes for leuvenair
fields_leuvenair = readJson('./leuvenair/LEUVENAIRmeta_final.json')
LAT_leuvenair = fields_leuvenair['LAT']
LON_leuvenair = fields_leuvenair['LON']

Note: sensors 8799 and 8827 are repeated in the json file!


In [3]:
# read all the sensor latitudes and longitudes for garmon
dframe = pd.read_csv('./garmon/GarmonData.csv', skiprows=0, nrows = None, usecols = ['LAT','LON'])
LAT_garmon = dframe.values[:,0]; LON_garmon = dframe.values[:,1]

# get only the unique values
LAT_garmon, index = np.unique(LAT_garmon.round(decimals=15), return_index=True)
LON_garmon = LON_garmon[index]

In [4]:
# read google maps API
with open('./leuvenair/apikey.txt') as f:
    my_api_key = f.readline()
    f.close

In [5]:
# Reference: https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html#basic-concepts

# get the base map
gmaps.configure(api_key=my_api_key) # Fill in with your API key

# zoom the map around the center of all leuvenair sensors
center_of_all_sensors = (np.mean(LAT_leuvenair),np.mean(LON_leuvenair))

# set the figure properties
figure_layout = {
    'width': '600px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# plot the base map
fobj = gmaps.figure(center=center_of_all_sensors, layout=figure_layout, zoom_level=13, map_type='TERRAIN')

# Note:
#'ROADMAP' is the default Google Maps style,
#'SATELLITE' is a simple satellite view,
#'HYBRID' is a satellite view with common features, such as roads and cities, overlaid,
#'TERRAIN' is a map that emphasizes terrain features.

In [6]:
# add leuvenair sensor layer on top of the base map
locations_leuvenair = list(zip(LAT_leuvenair,LON_leuvenair)) # provide the latitudes and longitudes
sensor_location_layer_leuvenair = gmaps.symbol_layer(locations_leuvenair, fill_color='red', stroke_color='red', scale=2)
fobj.add_layer(sensor_location_layer_leuvenair)
fobj

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [12]:
# add garmon sensor layer on top of the leuvenair map
locations_garmon = list(zip(LAT_garmon,LON_garmon)) # provide the latitudes and longitudes
GarmonLoc = np.array(locations_garmon)
maxLAT = np.max(GarmonLoc[:,0]); minLAT = np.min(GarmonLoc[:,0]);
maxLON = np.max(GarmonLoc[:,1]); minLON = np.min(GarmonLoc[:,1]);
print(np.array([[minLON, maxLON],[minLAT,maxLAT]]))
sensor_location_layer_garmon = gmaps.symbol_layer(locations_garmon, fill_color='green', stroke_color='green', scale=2)
fobj.add_layer(sensor_location_layer_garmon)
fobj

[[ 4.654269  4.789127]
 [50.846792 50.937804]]


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [36]:
# add telraam sensors
segments = requests.get("https://telraam-api.net/v0/segments/active")
data_segments = json_normalize(segments.json()['features'])
activeSegmentIDs = data_segments['properties.id'].values
segmentCoordinates = np.zeros((len(activeSegmentIDs),2))
# Read coordinates from each segment
for i in range(len(activeSegmentIDs)):
    urlSegment = 'https://telraam-api.net/v0/segments/id/'+str(activeSegmentIDs[i])
    active = requests.get(urlSegment)
    co = np.array(active.json()['features'][0]['geometry']['coordinates'][0]) # coordinates of the cameras
    segmentCoordinates[i,:] = np.mean(co, axis=0)

# create a new data frame for the active segments
activeSegmentIDs = np.expand_dims(np.array(activeSegmentIDs),axis=-1)
segmentCoordinates = np.concatenate((activeSegmentIDs,segmentCoordinates),axis=1)
segmentCoordinates = pd.DataFrame(data=segmentCoordinates, columns=['SegmentID', 'LON', 'LAT'])
segmentCoordinates = segmentCoordinates.astype({'SegmentID': 'uint64'})
segmentCoordinates.head(3)

,SegmentID,LON,LAT
0,1003073538,4.374151,50.867479
1,9000000005,4.672234,52.117404
2,3000136655,4.848457,50.326413


In [50]:
# remove segments outside Leuven
LeuvenCoord = np.array([[minLON, maxLON],[minLAT,maxLAT]])
print(LeuvenCoord)
LeuvenSegmentCoordinates = segmentCoordinates[segmentCoordinates['LON']>=LeuvenCoord[0,0]]
LeuvenSegmentCoordinates = LeuvenSegmentCoordinates[LeuvenSegmentCoordinates['LON']<=LeuvenCoord[0,1]]
LeuvenSegmentCoordinates = LeuvenSegmentCoordinates[LeuvenSegmentCoordinates['LAT']>=LeuvenCoord[1,0]]
LeuvenSegmentCoordinates = LeuvenSegmentCoordinates[LeuvenSegmentCoordinates['LAT']>=LeuvenCoord[1,1]]

print(LeuvenSegmentCoordinates.shape)
LeuvenSegmentCoordinates.head(3)

[[ 4.654269  4.789127]
 [50.846792 50.937804]]
(15, 3)


,SegmentID,LON,LAT
1,9000000005,4.672234,52.117404
43,178514,4.740499,51.186581
126,127699,4.691977,51.215608


In [45]:
columns_titles = ["LAT","LON"]
df=LeuvenSegmentCoordinates.reindex(columns=columns_titles)
locations_telraam = list(zip(df["LAT"].values,df["LON"].values))
print(locations_telraam[0])

(52.11740377719065, 4.672233870962212)


In [46]:
sensor_location_layer_telraam = gmaps.symbol_layer(locations_telraam, fill_color='blue', stroke_color='blue', scale=2)
fobj.add_layer(sensor_location_layer_telraam)
fobj

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…